# Introdction

Hey there, am happy to introduce a couple of tutorial notebooks to show you how to generate Images using deep learning. This is the very first notebook, I expect to create upto atleast three more of these while sharing diffent Image generation tips and ideas including (GANs and Diffusion Models). So this is the very first of the series and I will be covering the very basics of image generation and that is **auto-encoders**. so lets get to it.

# What are Auto Encoders ?

Auto Encoders lay the foundation of image generation. they are made up of majorly the encoder and decoder. the encoder as the name says encode an image(convert an image into latent space representation of that image) and the decoder converts that particular image from it;s latent space representation back to the original image(or something close to the original image).

![auto encoder](https://th.bing.com/th/id/R.ed5d36cdf7bdeca3cf4660ade4d544ae?rik=v0rvzTfPBrR58w&pid=ImgRaw&r=0)

So as illustrated in the above image, auto-encoders are quite powerful and can even be used in applications like denoising images plus also image generation. 

Anyways enough of the theory let's get to it 

# Import Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import init_notebook_mode
from tensorflow.keras.models import Model
from PIL import Image
from tqdm import tqdm

In [ ]:
data_path = "/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba"

In [ ]:
img_size = 64
batch_size = 64

# Load the Data

In [ ]:
#dataset contaiins over 200l images, will be using just the 50k for training, and 10k for testing

In [ ]:
def load_dataset():
    # Load the entire dataset into memory
    X = []
    
    pbar = tqdm(os.listdir(data_path)[:60000])
    for filename in pbar:
        img_path = os.path.join(data_path, filename)
        
        img = Image.open(img_path).convert('RGB').resize((img_size, img_size))
        img_array = np.asarray(img) / 255.
        X.append(img_array)

    return np.stack(X)

X = load_dataset()

In [ ]:
images = X[:15]

plt.figure(figsize = (20 , 20))
for i in range(15):
    plt.subplot(5 , 5, i+1)
    plt.subplots_adjust(hspace = 0.5 , wspace = 0.3)
    plt.imshow(images[i])
    plt.axis('off')

In [ ]:
train = X[:50000]
test = X[50000:]

In [ ]:
train.shape, test.shape

# Create the Encoder

In [ ]:
from keras import layers
from tensorflow import keras

In [ ]:
encoder_input = layers.Input(
    shape = (img_size, img_size, 3), name = "encoder_input"
)

x = layers.Conv2D(img_size, (3, 3), strides = 2, activation = 'relu', padding="same")(
 encoder_input
)
x = layers.Conv2D(128, (3, 3), strides = 2, activation = 'relu', padding="same")(x)
x = layers.Conv2D(512, (3, 3), strides = 2, activation = 'relu', padding="same")(x)
layer_shape = keras.backend.int_shape(x)
x = layers.Flatten()(x)
encoder_output = layers.Dense(10, name="encoder_output")(x)
encoder = Model(encoder_input, encoder_output)

In [ ]:
encoder.summary()

In [ ]:
layer_shape = layer_shape[1:]
layer_shape

# Create the Decoder

In [ ]:
decoder_input  = layers.Input(shape = (10,), name = "decoder_input")

x = layers.Dense(np.prod(layer_shape))(decoder_input)
x = layers.Reshape(layer_shape)(x)
x = layers.Conv2DTranspose(512, (3, 3), strides = 2, activation = 'relu', padding = "same")(x)
x = layers.Conv2DTranspose(128, (3, 3), strides = 2, activation = 'relu', padding = "same")(x)
x = layers.Conv2DTranspose(64, (3, 3), strides = 2, activation = 'relu', padding = "same")(x)

decoder_output = layers.Conv2D(
 3,
 (3, 3),
 strides = 1,
 activation="sigmoid",
 padding="same",
 name="decoder_output"
)(x)


decoder = Model(decoder_input, decoder_output)

In [ ]:
decoder.summary()

# Combine the two Models

In [ ]:
autoencoder = Model(encoder_input, decoder(encoder_output))

In [ ]:
autoencoder.summary()

In [ ]:
def root_mean_squared_error(y_true, y_pred):
    return keras.backend.sqrt(keras.backend.mean(keras.backend.square(y_pred - y_true)))

In [ ]:
autoencoder.compile(optimizer="adam", loss=root_mean_squared_error)

In [ ]:
# Train the autoencoder
autoencoder.fit(
    train,
    train,
    epochs=15,
    shuffle=True,
    validation_data = (test, test)
)

# Test the Model

In [ ]:
predictions = autoencoder.predict(test)

In [ ]:
images = test[:15]

plt.figure(figsize = (20 , 20))
for i in range(15):
    plt.subplot(5 , 5, i+1)
    plt.subplots_adjust(hspace = 0.5 , wspace = 0.3)
    plt.imshow(images[i])
    plt.axis('off')

In [ ]:
images = predictions[:15]

plt.figure(figsize = (20 , 20))
for i in range(15):
    plt.subplot(5 , 5, i+1)
    plt.subplots_adjust(hspace = 0.5 , wspace = 0.3)
    plt.imshow(images[i])
    plt.axis('off')

In [ ]:
# try to generate from random noise
noise_array = np.random.uniform(-2., 2., (1, 10)).astype('float32')
print(noise_array)

In [ ]:
pred = decoder.predict(noise_array)

plt.imshow((pred[0]))

And there we have our very own image generator 🥳. This is a very basic model but these are series of notebooks so we are definetly going to make this better

# Conclusion

So the predicted images are very blury but atleast for some the images, you can tell that they are for a human being and the hair color is some what consistent for all the images. The images can still be definelty improved by increasing on the latent space but we don't the model to end up predicting exactly what is in the dataset. <br>
Anyways this is a start and in the next notebook we will be implementing the famous **VAE** and we see how better it performs compared to the normal auto-encoders.

If you found this notebook helpful feel free to give me an upvote. otherwise. see you in the next Notebook